# 引言

本项目为帮助初学者入门profile工作所编写，是本组针对主流的profile工具所设计和编写的，包括一些小型模型以测试对于profiling的理解。如有不周到之处请谅解，我们也将不断完善该文档。

# 1. NVIDIA Nsight Compute

## 1.1 介绍

您如果需要安装NVIDIA Nsight，可以在https://developer.nvidia.com/tools-overview/nsight-compute/get-started，网址上获取安装包进行安装，安装流程省略。Nsight Compute 工具将其测量库插入到应用程序进程中，这允许分析器拦截与CUDA用户模式驱动程序的通信。此外，当检测到内核启动时，库可以从GPU收集所请求的性能指标。然后将结果传输回前端。Nsight Compute主要是针对CUDA Kernel较少的程序设计的，适合小型程序的profiling工作。

## 1.2 ncu常用指令

最直接且常用的用法是下面两个命令，如想了解详细解释，可见各细节部分<br>
ncu --set full -o profile_result ./application<br>
ncu --set default -o profile_result python your_script.py

### 1.2.1 基础性能分析

<b>ncu --set full -o profile_result ./application</b> <br>
<br>
<b>--set参数</b>：<br>default（基础性能概览：包含内核启动统计、占用率和GPU吞吐量等约35个核心指标，分析开销较小，适合初步性能评估）<br>detailed（深入性能分析：在default基础上增加了指令统计、内存工作负载分析和Warp状态统计等，提供约157个更详细的指标，用于深入探究性能瓶颈根源）<br>full（全面数据收集：提供最全面的指标（约162个），包含更详尽的内存工作负载图表和数据表。对系统开销最大，通常用于深度优化）<br>source（源代码关联分析：专注于约47个与源代码映射相关的性能计数器，需要编译时使用-lineinfo等选项生成调试信息，便于进行指令级优化）<br>
<br>
<b>-o参数</b>：profile_result为输出报告文件的名字 <br>
<br>
<b>./application</b>为需要执行的程序 <br>
<br>
完成后会在服务器上产生一个名称为profile_result.ncu-rep的文件，可以在本地用NVIDIA Nsight Compute打开 <br>

### 1.2.2 指标收集

<b>ncu --metrics dram__bytes.sum.per_second -o profile_result ./application</b><br>
<br>
<b>--metrics参数</b>：用以收集特定的指标，可以使用逗号分隔不同指标或使用正则表达式<br>
<br>
可以使用ncu --query-metrics命令查询所有可用的指标。<br>
<br>
常用的指标有：<br>
sm__cycles_active.avg.pct_of_peak_sustained_elapsed，SM活跃周期占比<br>
sm__warps_active.avg.pct，活跃Warp百分比<br>
dram__bytes.sum.per_second，显存带宽使用率<br>
smsp__sass_thread_inst_executed_op_fadd_pred_on.sum，已执行的特定类型指令（如浮点加法）数量<br>
Occupancy，理论占用率与实际占用率差距过大常表明工作负载不均衡<br>
l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum，全局内存加载请求数<br>
l1tex__t_sectors_pipe_lsu_mem_global_op_st.sum，全局内存存储请求数，分析缓存局部性<br>
<br>
第一行的命令完成后即收集完成内存带宽指标，结果输出到指定文件当中

### 1.2.3 结果过滤与聚焦

<b>ncu -k "my_kernel" -o profile_result ./application</b><br>
<br>
“my_kernel”处指定需要详细分析的内核函数，如何获得内核名称？<br>
直接运行一次 ncu (例如使用 --set default)，其输出的 "Section: Compute Workload Analysis" 部分会列出所有捕获到的内核名称，下面也会讲Compute Workload Analysis。您也可以使用正则表达式匹配多个kernel<br>
<br>
最终输出的报告专注于你筛选出的内核。例如，如果你使用 -k "vectorAdd"，那么报告里你只会看到名为 vectorAdd 的内核的详细性能分析；如果你的程序多次启动了同一个内核，Nsight Compute 默认可能会捕获多次启动的数据。最终的报告通常会汇总这些多次执行的数据（例如平均值），或者在时间线中展示多次调用。

### 1.2.4 输出与控制

<b>ncu --csv -o report.csv ./application</b><br>
<br>
使用 ncu --csv -o report.csv ./application 命令后，得到的 CSV 文件内容会直接反映 Nsight Compute 的性能分析结果。<br>
各列通常包括：<br>
内核标识信息：例如 ID，Process ID，Process Name，Kernel Name，Context ID 等，用于区分不同的内核和其启动实例。<br>
性能指标数据：核心的性能指标列，其名称直接来自您使用 --metrics 指定的指标，或所选 --set 规则集包含的指标。例如 sm__cycles_active.avg.pct (SM 活跃周期百分比)，dram__bytes.sum.per_second (显存带宽)，l1tex__t_sectors_pipe_lsu_mem_global_op_ld.sum (全局加载请求数) 等。<br>
其他元数据：可能包括 Grid Size，Block Size，Registers Per Thread，Static Shared Memory 等内核配置信息。<br>

### 1.2.5 如何使用ncu运行某个python文件？

<b>ncu -o my_profile python your_script.py</b><br>
以上是基本用法，若需要其他更多的参数进行profile工作，可类比上述参数添加更多的分析

## 1.3 界面介绍

![界面总览](./images/view.png)

上图为界面的总览图，只需要将在服务器上生成的ncu文件下载后，拖动到NVIDIA Nsight Compute界面上，即可获知详细的profile报告

### 1.3.1 Summary

![Summary概览图](./images/summary.png)

序号 0-8 则是依次运算的kernel，其信息包括：<br>
ID: 每个函数的唯一标识符。<br>
Estimated Speedup: 估计的加速比，表示如果优化这个函数可能带来的速度提升。<br>
Function Name: 函数的名称。<br>
Demangled Name: 去掉修饰符的函数名称。<br>
Duration: 函数执行时间（以ns为单位）。<br>
Runtime Improvement: 估计的运行时间提示（以ns为单位），表示如果优化这个函数可能带来的运行时间提升。<br>
Compute Throughput: 计算吞吐量。SM 吞吐量假设在 SMSPs 间负载平衡理想的情况下 （此吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续率的百分比）。<br>
Memory Throughput: 内存吞吐量。计算内存管道吞吐量 （此吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续率的百分比）。<br>
Registers: 每个线程使用的寄存器数量。<br>
GridSize：kernel启动的网格大小。<br>
BlockSize：每个Block的线程数。<br>
Cycles：GPC指令周期。GPC：通用处理集群（General Processing Cluster）包含以 TPC（纹理处理集群）形式存在的 SM、纹理和 L1 缓存。 它在芯片上被多次复制。<br>
最上部的Result默认显示的是ID=0的 kernel 运行的部分信息，包括GPU型号及频率<br>
并且，从该图可以初步得到一些信息，比如大概率memory-bound ID8，大概率compute-bound ID7

### 1.3.2 GPU Speed Of Light Throughput

![throughput-1](./images/throughput-1.png)

左侧指标可以详细看到 Compute 和 不同层次的 Memory 的实际利用效率的情况，由此可以定位其在 roofline 中的位置。<br>
从这个结果可以看出：<br>
内存吞吐量(40.27%)高于计算吞吐量(32.70%)，表明这可能是一个内存密集型任务。<br>
L2 缓存和 DRAM 吞吐量相对较低，可能存在优化空间。<br>
L1吞吐量与总体内存吞吐量相近，说明主要的内存操作与该部分交互，需要特别说明的是 Shared memory 也统计在内。<br>
<br>
右侧指标说明了执行时间、总的周期数、SM活跃的周期数以及SM和DRAM的频率

![throughput-2](./images/throughput-2.png)

![throughput-3](./images/throughput-3.png)

最后这部分是roofiline model的一个绘制，绿色的点在斜线下面，可以很直观地判断出该kernel是memory bound的一个内核

### 1.3.3 Memory Workload Analysis

![memory-1](./images/memory-1.png)

该指标为内存资源的使用情况，主要包括通信带宽、内存指令的最大吞吐量。详细的数据表如上：<br>
<br>
Memory Throughput: 34.63 Gbyte/s<br>
即每秒在DRAM中访问的字节数。<br>
<br>
L1/TEX Hit Rate: 74.01%<br>
每个 sector 的 sector 命中次数 （这个比率指标表示跨所有子单元实例的值，以百分比表示）。<br>
<br>
l1tex：一级（L1）/纹理缓存位于GPC内部。 它可以用作定向映射的共享内存和/或在其缓存部分存储全局、本地和纹理数据。<br>
<br>
sector：缓存线或设备内存中对齐的32字节内存块。 一个L1或L2缓存线是四个sector，即128字节。 如果标签存在且sector数据在缓存线内，则sector访问被归类为命中。 标签未命中和标签命中但数据未命中都被归类为未命中。<br>
<br>
L2 Hit Rate: 87.19%<br>
L2sector查找命中的比例 （这个比率指标表示跨所有子单元实例的值，以百分比表示）。<br>
<br>
l2s：二级（L2）缓存切片是二级缓存的一个子分区。 l2s_t 指的是其标签阶段。 l2s_m 指的是其未命中阶段。 l2s_d 指的是其数据阶段。<br>
<br>
Mem Busy: 40.27%<br>
缓存和DRAM内部活动的吞吐量（这个吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续速率的百分比）<br>
<br>
Max Bandwidth: 17.53%<br>
SM<->缓存<->DRAM之间互连的吞吐量 （这个吞吐量指标表示在所有子单元实例的经过周期内达到的峰值持续速率的百分比）<br>
<br>
L2 Compression Ratio: 0
L2 Compression Success Rate: 0

![memory-2](./images/memory-2.png)

上图显示了各级 memory 的连接关系及使用情况，整体情况一目了然。<br>

![memory-3](./images/memory-3.png)

在上图中，我们主要关心load和store的情况

![memory-4](./images/memory-4.png)

在上图中，我们也主要关心load和store的情况

### 1.3.4 Compute Workload Analysis

![compute-1](./images/compute-1.png)

分析完了内存的情况后，接下来分析计算单元的使用情况。即对流式多处理器（SM）的计算资源进行详细分析，包括实际达到的每时钟周期指令数（IPC）以及每个可用流水线的利用率。主要指标包括：<br>
<br>
Executed Ipc Elapsed: 1.30 inst/cycle<br>
执行的warp指令数，此计数器指标表示所有子单元实例中每个执行周期的平均操作数。<br>
<br>
Executed Ipc Active: 1.72 inst/cycle<br>
执行的warp指令数，此计数器指标表示所有子单元实例中每个活动周期的平均操作数。<br>
<br>
Issued Ipc Active: 1.73 inst/cycle<br>
发出的warp指令数，此计数器指标表示所有子单元实例中每个活动周期的平均操作数。与上一项比较可知，在活动周期内发出的指令都被执行。<br>
<br>
SM Busy<br>
假设SMSP间理想负载平衡的SM核心指令吞吐量，此吞吐量指标表示在所有子单元实例的活动周期内达到的峰值持续率的百分比。<br>
<br>
SMSPs: 每个SM被划分为四个处理块，称为SM子分区。 SM子分区是SM上的主要处理元素。 一个子分区管理固定大小的warp池。<br>
<br>
Issue Slots Busy<br>
发出的warp指令数，此计数器指标表示在所有子单元实例的活动周期内达到的峰值持续率的平均百分比

![compute-2](./images/compute-2.png)

接下来是一些主要计算单元的利用率情况，先分别介绍一下这些计算单元：<br>

FMA: Fused Multiply Add/Accumulate，融合乘加。FMA流水线处理大多数FP 32算法（FADD、FMUL、FMAD）。它还执行整数乘法运算（IMUL、IMAD）以及整数点积。<br>
ALU: Arithmetic Logic Unit, 算术逻辑单元。ALU负责执行大多数位操作和逻辑指令。它还执行整数指令，不包括IMAD和IMUL。在NVIDIA Ampere架构芯片上，ALU流水线执行快速的FP 32到FP 16转换。<br>
LSU: Load Store Unit, 加载存储单元。LSU流水线向L1 TEX单元发出用于全局、本地和共享内存的加载、存储、原子和归约指令。它还向L1 TEX单元发出特殊的寄存器读取（S2 R）、混洗和CTA级到达/等待屏障指令。<br>
TMA: Tensor Memory Access Unit, 张量存储器访问单元。在全局内存和共享内存之间提供高效的数据传输机制，能够理解和遍历多维数据布局。<br>
ADU: Address Divergence Unit, 地址分支单元。ADU负责分支/跳转的地址发散处理。它还支持恒定加载和块级屏障指令。<br>
CBU：Convergence Barrier Unit，汇聚屏障单元。CBU负责曲速级收敛、屏障和分支指令。<br>
TEX: Texture Unit, 纹理单元。SM纹理流水线将纹理和表面指令转发到L1TEX单元的TEXIN阶段。在FP64或Tensor流水线解耦的GPU上，纹理流水线也会转发这些类型的指令。<br>
Uniform: Uniform Data Path, 统一数据路径。这个标量单元执行所有线程使用相同输入并生成相同输出的指令。<br>
XU: Transcendental and Data Type Conversion Unit, 超越和数据类型转换单元。XU管道负责特殊函数，如sin、cos和倒数平方根。它还负责int到float和float到int类型的转换.<br>

### 1.3.5 Statistics

![statistic-1](./images/statistic-1.png)

分析所有 warp 在内核执行期间所花费的周期数。warp state 描述 warp 是否准备好发出下一个指令。每条指令的 warp 周期定义了两条连续指令之间的延迟。该值越高，隐藏此延迟所需的 warp 并行度就越高。对于每个warp state，该图表显示了每个发出的指令在该状态下花费的平均周期数。stall 并不总是影响整体性能，也不是完全可以避免的。<br>
<br>
平均而言，该内核的每个warp在等待微调度器选择要发出的warp时会花费18.57个周期。未被选中的warp是指在该周期内没有被调度器选择发出的符合条件的warp。大量未被选中的warp通常意味着有足够多的warps来覆盖warp延迟，并且可以考虑减少活跃warps数量以可能增加缓存一致性和数据局部性。<br>

![statistic-2](./images/statistic-2.png)

![statistic-3](./images/statistic-3.png)

### 1.3.6 Occupancy

占用率(Occupancy)是指每个SM上活跃线程组(warp)的数量与可能的最大活跃线程组数量的比率。另一种看待占用率的方式是,它表示硬件处理线程组的能力中实际被使用的百分比。虽然较高的占用率并不总能带来更高的性能,但是低占用率会降低隐藏延迟的能力,从而导致整体性能下降。在执行过程中,理论占用率和实际达到的占用率之间存在较大差异,通常表示工作负载高度不均衡。占用率反映了GPU资源的利用情况,是评估CUDA程序性能的一个关键指标。过低的占用率会导致性能下降,需要分析并优化造成低占用率的原因。

![occupancy-1](./images/occupancy-1.png)

![occupancy-2](./images/occupancy-2.png)

## 1.4 程序示例

所有示例程序可见目录./nvidia_nsight/nsight_compute，您可以自行选择需要运行的程序，并使用ncu命令运行，然后进行一些简单的profile工作。

## 1.5 参考文献

https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#roofline<br>
https://zhuanlan.zhihu.com/p/715022552<br>
https://github.com/ifromeast/cuda_learning/blob/main/05_cuda_mode/ncu_profile/readme.md


# 2. Pytorch Profiler

## 2.1 介绍

## 2.2 常用指令

## 2.3 界面介绍

## 2.4 程序示例

## 2.5 参考文献

# 3. NVIDIA Nsight System

## 3.1 介绍

## 3.2 常用指令

## 3.3 界面介绍

## 3.4 程序示例

## 3.5 参考文献